![resources/images/dhlab-logo-nb.png](../resources/images/dhlab-logo-nb.png)

# Frekvenslister
Frekvenslister fra tekster kan danne utgangspunkt for mye informasjon om teksten.

Her bruker vi klassen `Counts` for å hente frekvensklister (absoluttfrekvenser) fra boka.

`urn`er identifikator for boka i Nasjonalbibliotekets samling. Under eksempelfila om korpusbygging() finnes mer informasjon om hvordan man bygger et korpus. Urn finner man i bokas metadata, og de er på denne formen: https://urn.nb.no/URN:NBN:no-nb_digibok_2012051608012. Det er de 13 sifrene på slutten vi bruker her.

In [23]:
from dhlab import Counts, totals, Corpus
import dhlab.graph_networkx_louvain as gnl
import pandas as pd
import dhlab.nbtext as nb

### Frekvensliste for enkeltbøker

#### Kort sakprosabok

In [24]:
urn = "URN:NBN:no-nb_digibok_2021011848587"
bok = Counts([urn])

In [25]:
bok.frame.iloc[:, 0]

,                184
.                162
og                94
i                 93
det               51
                ... 
avbetaling         1
atenerinnene       1
1942               1
april              1
filmversjonen      1
Name: 100431574, Length: 1597, dtype: int64

Vi kan sammenlikne frekvenslista for boka med et større korpus, som vi her legger i variabelen tot

In [26]:
tot = totals()

Vi lager en dataramme, `nb.frame()` for forholdet mellom frekvensene for ordene i vann og for de 50000 mest frekvente ordene i hele nb.no

In [27]:
(bok.frame.iloc[:, 0] / tot.iloc[:, 0]).sort_values(ascending=False).head(20)

Hedda          0.000050
def            0.000030
Ivo            0.000026
Skjønberg      0.000023
Claes          0.000019
Gill           0.000017
oppførelse     0.000016
teatrene       0.000016
VAN            0.000015
væsen          0.000013
Alten          0.000013
Rosenkrantz    0.000011
Kaas           0.000011
poetiske       0.000010
ensemblet      0.000010
Høst           0.000010
Jæger          0.000010
kontra         0.000009
Peggy          0.000009
Vauxhall       0.000009
dtype: float64

In [28]:
tot

,freq
.,7655423257
",",5052171514
i,2531262027
og,2520268056
-,1314451583
...,...
tidspunkter,110667
dirigenter,110660
ondartet,110652
kulturtilbud,110652


#### Knausgårds *Min kamp / Første bok*

In [29]:
min_kamp_1 = Counts(["URN:NBN:no-nb_digibok_2014032405041"])
min_kamp_1.frame.head(20)

,100197175
",",14332
.,9480
og,5383
det,4430
jeg,4175
i,3657
var,3238
på,3152
-,2500
som,2463


In [30]:
(min_kamp_1.frame.iloc[:, 0] / tot.iloc[:,0]).sort_values(ascending=False).head(20).to_frame("ratio")

,ratio
Yngve,0.000572
Farmor,0.000376
pappas,0.000231
farmor,0.000203
Ylva,0.000190
skrudde,0.000154
Jada,0.000130
Fløgstad,0.000119
helte,0.000114
koppen,0.000108


### Sammenlikning av frekvens for ulike bøker

In [31]:
# Frekvenslisiter for 20 Anne-Cath. Vestly bøker
vestly = Counts(
    Corpus(
        author="Anne*Vestly*",
        limit=20
    )
)
vestly.frame.head()

,100576702,100461961,100475774,100512518,100007325,100559117,100554153,100045850,100133228,100129179,100152855,100155158,100495412,100265579,100268749,100303061,100429142,100429382,100592369,100602349
",",2927.0,4457.0,4848.0,2398.0,2944.0,2282.0,1881.0,1750.0,2547.0,10898.0,2901.0,2724.0,578.0,2559.0,31.0,68.0,2744.0,1863.0,293.0,2265.0
.,2676.0,3654.0,3600.0,1946.0,2440.0,2087.0,1743.0,1848.0,2258.0,10085.0,2227.0,1954.0,688.0,2237.0,56.0,87.0,2312.0,1886.0,259.0,1704.0
og,1642.0,2301.0,3539.0,1316.0,1450.0,1597.0,923.0,923.0,1301.0,6955.0,1453.0,1259.0,466.0,1301.0,13.0,24.0,1616.0,878.0,155.0,1173.0
»,1400.0,2527.0,2157.0,1219.0,1810.0,985.0,1014.0,955.0,1436.0,4908.0,1436.0,1409.0,3.0,1438.0,23.0,7.0,1299.0,1020.0,6.0,1008.0
«,1399.0,2532.0,2151.0,1216.0,1808.0,983.0,1010.0,957.0,1426.0,4910.0,1436.0,1407.0,3.0,1435.0,19.0,14.0,1300.0,1024.0,5.0,1011.0


In [32]:
# Vi kan legge på et varmekart for lettere å få oversikt

nb.heatmap(vestly.frame.head(20))

,100576702,100461961,100475774,100512518,100007325,100559117,100554153,100045850,100133228,100129179,100152855,100155158,100495412,100265579,100268749,100303061,100429142,100429382,100592369,100602349
",",2927.000000,4457.000000,4848.000000,2398.000000,2944.000000,2282.000000,1881.000000,1750.000000,2547.000000,10898.000000,2901.000000,2724.000000,578.000000,2559.000000,31.000000,68.000000,2744.000000,1863.000000,293.000000,2265.000000
.,2676.000000,3654.000000,3600.000000,1946.000000,2440.000000,2087.000000,1743.000000,1848.000000,2258.000000,10085.000000,2227.000000,1954.000000,688.000000,2237.000000,56.000000,87.000000,2312.000000,1886.000000,259.000000,1704.000000
og,1642.000000,2301.000000,3539.000000,1316.000000,1450.000000,1597.000000,923.000000,923.000000,1301.000000,6955.000000,1453.000000,1259.000000,466.000000,1301.000000,13.000000,24.000000,1616.000000,878.000000,155.000000,1173.000000
»,1400.000000,2527.000000,2157.000000,1219.000000,1810.000000,985.000000,1014.000000,955.000000,1436.000000,4908.000000,1436.000000,1409.000000,3.000000,1438.000000,23.000000,7.000000,1299.000000,1020.000000,6.000000,1008.000000
«,1399.000000,2532.000000,2151.000000,1216.000000,1808.000000,983.000000,1010.000000,957.000000,1426.000000,4910.000000,1436.000000,1407.000000,3.000000,1435.000000,19.000000,14.000000,1300.000000,1024.000000,5.000000,1011.000000
det,927.000000,1505.000000,1772.000000,789.000000,890.000000,863.000000,599.000000,687.000000,790.000000,4070.000000,811.000000,728.000000,196.000000,791.000000,9.000000,6.000000,950.000000,630.000000,89.000000,733.000000
sa,898.000000,1670.000000,1451.000000,820.000000,1195.000000,650.000000,567.000000,602.000000,944.000000,3091.000000,918.000000,917.000000,79.000000,943.000000,5.000000,7.000000,870.000000,613.000000,0.000000,672.000000
så,785.000000,1128.000000,1096.000000,585.000000,744.000000,633.000000,588.000000,455.000000,645.000000,2896.000000,714.000000,658.000000,117.000000,647.000000,6.000000,3.000000,729.000000,498.000000,73.000000,616.000000
på,674.000000,988.000000,1157.000000,555.000000,618.000000,641.000000,487.000000,418.000000,603.000000,2921.000000,632.000000,557.000000,146.000000,604.000000,5.000000,5.000000,649.000000,492.000000,65.000000,495.000000
var,647.000000,1004.000000,1213.000000,621.000000,540.000000,660.000000,351.000000,464.000000,631.000000,2974.000000,639.000000,490.000000,81.000000,633.000000,7.000000,4.000000,692.000000,426.000000,7.000000,630.000000


### Frekvens for spesifikke termer

In [33]:
def heatmap_loc(dr, ordliste, sorter=0):
    df = dr.loc[ordliste]
    return nb.heatmap(df.sort_values(by=df.columns[sorter], ascending=False))

In [34]:
# Her lager vi en funksjon heatmap_loc som vi bruker for å plukke ut enkelte ord fra korpuset som vi vil sammenlikne.


heatmap_loc(vestly.frame, "Mormor Morten Marte Ole Aleksander Guro Lillebror Knerten".split())


,100576702,100461961,100475774,100512518,100007325,100559117,100554153,100045850,100133228,100129179,100152855,100155158,100495412,100265579,100268749,100303061,100429142,100429382,100592369,100602349
Morten,145.000000,0.000000,34.000000,22.000000,1.000000,148.000000,0.000000,0.000000,0.000000,897.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000
Marte,43.000000,0.000000,4.000000,11.000000,1.000000,73.000000,0.000000,0.000000,0.000000,361.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
Mormor,34.000000,0.000000,113.000000,29.000000,1.000000,66.000000,0.000000,0.000000,0.000000,265.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,2.000000
Ole,0.000000,0.000000,0.000000,0.000000,10.000000,7.000000,479.000000,450.000000,2.000000,114.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,450.000000,0.000000,1.000000
Aleksander,0.000000,0.000000,0.000000,0.000000,10.000000,7.000000,479.000000,448.000000,0.000000,105.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,443.000000,0.000000,1.000000
Guro,0.000000,0.000000,93.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Lillebror,0.000000,1087.000000,7.000000,2.000000,767.000000,0.000000,0.000000,0.000000,664.000000,0.000000,711.000000,669.000000,0.000000,662.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Knerten,0.000000,643.000000,10.000000,2.000000,394.000000,0.000000,0.000000,0.000000,307.000000,0.000000,329.000000,300.000000,0.000000,299.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [35]:
# Er forfatteren konsekvent når det kommer til ordformer?

heatmap_loc(vestly.frame, "boken boka sola solen".split())

,100576702,100461961,100475774,100512518,100007325,100559117,100554153,100045850,100133228,100129179,100152855,100155158,100495412,100265579,100268749,100303061,100429142,100429382,100592369,100602349
boka,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
solen,1.000000,1.000000,5.000000,0.000000,8.000000,0.000000,0.000000,1.000000,1.000000,7.000000,0.000000,2.000000,1.000000,1.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000
boken,0.000000,3.000000,0.000000,3.000000,3.000000,1.000000,0.000000,0.000000,8.000000,4.000000,0.000000,7.000000,0.000000,7.000000,0.000000,0.000000,17.000000,2.000000,0.000000,2.000000
sola,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
